In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

products = [f"PROD{i:03}" for i in range(1, 21)]
categories = ['Shirt','Shoes','Bag','Jeans','Dress']
users = [f"USER{i:03}" for i in range(1, 11)]

data = []

for user in users:
    for product in products:
        category = np.random.choice(categories)
        description = f"This is a {category} with unique style."
        price = np.random.choice([499, 699, 999, 1299])
        discount = np.random.choice([0, 10, 20])
        rating = np.random.randint(1,6)
        sales_last_month = np.random.randint(5,50)
        data.append([user, product, category, description, price, discount, rating, sales_last_month])

df = pd.DataFrame(data, columns=['user_id','product_id','category','description','price','discount','rating','sales_last_month'])
df.to_csv("fashion_trend_data.csv", index=False)


In [ ]:
from sentence_transformers import SentenceTransformer

model_embed = SentenceTransformer('all-MiniLM-L6-v2')
df['desc_embed'] = df['description'].apply(lambda x: model_embed.encode(x))


In [ ]:
top_trending = df.groupby('product_id')['sales_last_month'].sum().sort_values(ascending=False).head(10)


In [ ]:
from lightfm import LightFM
from lightfm.data import Dataset

# Build interaction matrix
dataset = Dataset()
dataset.fit(df['user_id'], df['product_id'])
(interactions, weights) = dataset.build_interactions([(row['user_id'], row['product_id'], row['rating']) for _, row in df.iterrows()])

# Train LightFM model
model = LightFM(loss='warp')
model.fit(interactions, epochs=20, num_threads=2)


In [ ]:
def recommend_products(model, dataset, user_id, n=5):
    n_users, n_items = interactions.shape
    user_x = dataset.mapping()[0][user_id]
    scores = model.predict(user_x, np.arange(n_items))
    top_items = np.argsort(-scores)[:n]
    id_map = {v:k for k,v in dataset.mapping()[2].items()}  # item mapping
    return [id_map[i] for i in top_items]

recommend_products(model, dataset, 'USER001')
